<a href="https://colab.research.google.com/github/joshuwaifo/A-Bible-Pre-trained-Transformer-Model/blob/main/TransformerBlock_ResidualConnection_BibleGPT_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Previously on BibleGPT 1-5

1 minute 2 seconds run time CPU Google Colab

In [1]:
!wget https://raw.githubusercontent.com/tushortz/variety-bible-text/master/bibles/nasb.txt

import torch
import torch.nn as nn
from torch.nn import functional as F

batch_size = 32
block_size = 8
max_iters = 5000
eval_interval = 300
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 32

torch.manual_seed(1337)

with open('nasb.txt', 'r', encoding='utf-8') as f:
  text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
n_train = int(0.64*len(data))
n_val = int(0.8*len(data))
train_data = data[:n_train]
val_data = data[n_train:n_val]
test_data = data[n_val:]

def get_batch(split):
  data = train_data if split == 'train' else val_data if split == 'val' else test_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x, y = x.to(device), y.to(device)
  return x,y

@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out


class Head(nn.Module):

  def __init__(self, head_size):
    super().__init__()

    self.key = nn.Linear(n_embd, head_size, bias=False)
    self.query = nn.Linear(n_embd, head_size, bias=False)
    self.value = nn.Linear(n_embd, head_size, bias=False)

    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

  def forward(self, x):
    B,T,C = x.shape
    k = self.key(x)
    q = self.key(x)
    wei = q @ k.transpose(-2, -1) * C**0.5
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
    wei = F.softmax(wei, dim=-1)
    v = self.value(x)
    out = wei @ v
    return out

class BigramLanguageModel(nn.Module):

  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    self.sa_head = Head(n_embd)
    self.lm_head = nn.Linear(n_embd, vocab_size)


  def forward(self, idx, targets=None):
    B, T = idx.shape
    tok_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T, device=device))
    x = tok_emb + pos_emb
    x = self.sa_head(x)
    logits = self.lm_head(x)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      idx_cond = idx[:, -block_size:]
      logits, loss = self(idx_cond)
      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1)
      idx = torch.cat((idx, idx_next), dim=1)
    return idx


model = BigramLanguageModel()
m = model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

  if iter % eval_interval == 0:
    losses = estimate_loss()

  xb, yb = get_batch('train')

  logits, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()

context = torch.zeros((1,1), dtype=torch.long, device=device)


--2024-08-10 12:41:08--  https://raw.githubusercontent.com/tushortz/variety-bible-text/master/bibles/nasb.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4685837 (4.5M) [text/plain]
Saving to: ‘nasb.txt’

nasb.txt            100%[===================>]   4.47M  --.-KB/s    in 0.02s   

2024-08-10 12:41:08 (196 MB/s) - ‘nasb.txt’ saved [4685837/4685837]



Today on BibleGPT 6

Train below

See what scores we get to

In [ ]:
# !wget https://raw.githubusercontent.com/tushortz/variety-bible-text/master/bibles/nasb.txt

import torch
import torch.nn as nn
from torch.nn import functional as F

batch_size = 32
block_size = 8
max_iters = 5000 # increase number of iterations as the learning rate is lower
eval_interval = 300
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 32

torch.manual_seed(1337)

with open('nasb.txt', 'r', encoding='utf-8') as f:
  text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
n_train = int(0.64*len(data))
n_val = int(0.8*len(data))
train_data = data[:n_train]
val_data = data[n_train:n_val]
test_data = data[n_val:]

def get_batch(split):
  data = train_data if split == 'train' else val_data if split == 'val' else test_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x, y = x.to(device), y.to(device)
  return x,y

@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out


class Head(nn.Module):

  def __init__(self, head_size):
    super().__init__()

    self.key = nn.Linear(n_embd, head_size, bias=False)
    self.query = nn.Linear(n_embd, head_size, bias=False)
    self.value = nn.Linear(n_embd, head_size, bias=False)

    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

  def forward(self, x):
    B,T,C = x.shape
    k = self.key(x)
    q = self.key(x)
    wei = q @ k.transpose(-2, -1) * C**0.5
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
    wei = F.softmax(wei, dim=-1)
    v = self.value(x)
    out = wei @ v
    return out

class BigramLanguageModel(nn.Module):

  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    self.sa_head = Head(n_embd)
    self.lm_head = nn.Linear(n_embd, vocab_size)


  def forward(self, idx, targets=None):
    B, T = idx.shape
    tok_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T, device=device))
    x = tok_emb + pos_emb
    x = self.sa_head(x)
    logits = self.lm_head(x)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      idx_cond = idx[:, -block_size:]
      logits, loss = self(idx_cond)
      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1)
      idx = torch.cat((idx, idx_next), dim=1)
    return idx


model = BigramLanguageModel()
m = model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

  if iter % eval_interval == 0:
    losses = estimate_loss()

  xb, yb = get_batch('train')

  logits, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()

context = torch.zeros((1,1), dtype=torch.long, device=device)


Review:

Scaled product attention implemented

Now:

Multi-head attention:

- applying multiple attentions in parallel

- concatentating the results

In [ ]:
# multiple heads of self attention running in parallel
class MultiHeadAttention(nn.Module):
  """ Multiple heads of self-attention in parallel """


  # however multiple heads you want and whatever the head size is of each
  def  __init__(self, num_heads, head_size):
    super().__init__()
    # run all of them in parallel into a list
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])


  # concatenate all the outputs
  # concatenating over the channel dimension
  def forward(self, x):
    return torch.cat([h(x) for h in self.heads], dim=-1)

# updated Bigram language model
class BigramLanguageModel(nn.Module):

  # instead of 1 communication channel
  # we now have 4 communication channels
  def __init__(self):
    super().__init__()
    # each token directly reads off the logits for the next token from a lookup table
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)

    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    self.sa_heads = MultiHeadAttention(4, n_embd//4) # i.e. 4 heads of 8-dimensional self-attention
    # from each communication channel, going to gather eight dimensional vectors
    # there will be 4 of them
    # that concatenates to give us 32 which is the original n_embd
    # similar to (group) convolutions
    # instead of having 1 large convolution
    # we have convolutional groups
    # this is multi-headed self-attention

    self.lm_head = nn.Linear(n_embd, vocab_size)


  def forward(self, idx, targets=None):
    B, T = idx.shape
    tok_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T, device=device))
    x = tok_emb + pos_emb
    x = self.sa_heads(x)
    logits = self.lm_head(x)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      idx_cond = idx[:, -block_size:]
      logits, loss = self(idx_cond)
      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1)
      idx = torch.cat((idx, idx_next), dim=1)
    return idx


model = BigramLanguageModel()
m = model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

  if iter % eval_interval == 0:
    losses = estimate_loss()

  xb, yb = get_batch('train')

  logits, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()

context = torch.zeros((1,1), dtype=torch.long, device=device)


It helps to create multiple independent channels of communication

Where each is gathering lots of different types of data

Then decode the output


In [ ]:
# Paper attention is all you need 2017

# token encoding + position encoding

# encoder: (self) unmasked (not a triangular matrix based) multi-head attention
# decoder: (self) masked (triangular matrix based) multi-head attention

# cross (multi-head) attention

# (position-wise) feed forward network part = multi-layer perceptron

Adding computation into the network

In [ ]:
class FeedForward(nn.Module):
  """ a simple linear layer followed by a non-linearity """

  # little feedforward single layer
  # just a linear followed by a relu nonlinearity
  # applying linear on a per token level
  # all the tokens do this independently
  def __init__(self, n_embd):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embd, n_embd),
        nn.ReLU(),
    )

  # computation on a per node level
  def forward(self, x):
    return self.net(x)


# updated Bigram language model
class BigramLanguageModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)

    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    self.sa_heads = MultiHeadAttention(4, n_embd//4) # i.e. 4 heads of 8-dimensional self-attention
    self.ffwd = FeedForward(n_embd)
    self.lm_head = nn.Linear(n_embd, vocab_size)


  def forward(self, idx, targets=None):
    B, T = idx.shape
    tok_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T, device=device))
    x = tok_emb + pos_emb
    x = self.sa_heads(x)
    x = self.ffwd(x) # (B,T,C)
    logits = self.lm_head(x) # (B,T,vocab_size)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      idx_cond = idx[:, -block_size:]
      logits, loss = self(idx_cond)
      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1)
      idx = torch.cat((idx, idx_next), dim=1)
    return idx

Self attention is the communication

Once they've gathered all the data

They then think on the data individually

Now:

We are going to intersperse the communication (self attention) with the computation (feedforward)

This is what the transformer does, it has

- blocks that communicate and then compute

- groups such blocks and replicates them

In [ ]:
# Let's make the block
# the block is the right greyed part in the transformer-model architecutre without the cross attention

class Block(nn.Module):
  """ Transformer block: communication followed by computation """

  # the block basically intersperses communication and then computation
  def __init__(self, n_embd, n_head):
    # n_head is effectively like the group size in group convolution
    # n_embd: embedding dimension, n_head: the number of heads we'd like
    super().__init__()
    head_size = n_embd // n_head
    # communication done using multi-headed self-attention
    self.sa = MultiHeadAttention(n_head, head_size)
    # computation done using the feed forward network on all the tokens independently
    self.ffwd = FeedForward(n_embd)

  def forward(self, x):
    x = self.sa(x)
    x = self.ffwd(x)
    return x

# number of heads we'd like is 4
# because n_embd is 32
class BigramLanguageModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)

    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    # create blocks
    # interspersing communication and feedforward many times (3 in this example)
    self.blocks = nn.Sequential(
        Block(n_embd, n_head=4),
        Block(n_embd, n_head=4),
        Block(n_embd, n_head=4)
    )

    # self.sa_heads = MultiHeadAttention(4, n_embd//4) # i.e. 4 heads of 8-dimensional self-attention
    # self.ffwd = FeedForward(n_embd)
    self.lm_head = nn.Linear(n_embd, vocab_size)


  def forward(self, idx, targets=None):
    B, T = idx.shape
    tok_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T, device=device))
    x = tok_emb + pos_emb
    x = self.blocks(x)
    logits = self.lm_head(x) # (B,T,vocab_size)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      idx_cond = idx[:, -block_size:]
      logits, loss = self(idx_cond)
      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1)
      idx = torch.cat((idx, idx_next), dim=1)
    return idx

Head size becomes 8

With multiple blocks, neural network starts to scale up which makes optimisation quite challenging and performance affected

Let's make sure the networks remain optimisable

Methods:

- skip connections also known as residual connections (comes from the paper: Deep Residual Learning for Image Recognition, 2015): trans

Addition distributes gradients equally when backpropagating

Gradient distribution highway



In [ ]:
# Let's implement the skip connnection in the block

# Let's make the block
# the block is the right greyed part in the transformer-model architecutre without the cross attention

class Block(nn.Module):
  """ Transformer block: communication followed by computation """

  # the block basically intersperses communication and then computation
  def __init__(self, n_embd, n_head):
    # n_head is effectively like the group size in group convolution
    # n_embd: embedding dimension, n_head: the number of heads we'd like
    super().__init__()
    head_size = n_embd // n_head
    # communication done using multi-headed self-attention
    self.sa = MultiHeadAttention(n_head, head_size)
    # computation done using the feed forward network on all the tokens independently
    self.ffwd = FeedForward(n_embd)

  def forward(self, x):
    # skip connection or residual connection
    # x, it forks off to do some communication off the residual highway
    # then joins back to x again
    x = x + self.sa(x)
    # x, it forks off to do some computation off the residual highway
    # then joins back to x again
    x = x + self.ffwd(x)
    return x

class MultiHeadAttention(nn.Module):
  """ Multiple heads of self-attention in parallel """


  # however multiple heads you want and whatever the head size is of each
  def  __init__(self, num_heads, head_size):
    super().__init__()
    # run all of them in parallel into a list
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(n_embd, n_embd)

  # concatenate all the outputs
  # concatenating over the channel dimension
  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)
    # projection is just a linear transformation of the outcome
    # it becomes the projection back into the residual pathway
    out = self.proj(out)
    return

class FeedForward(nn.Module):
  """ a simple linear layer followed by a non-linearity """

  # little feedforward single layer
  # just a linear followed by a relu nonlinearity
  # applying linear on a per token level
  # all the tokens do this independently
  def __init__(self, n_embd):
    super().__init__()
    # add a linear projection
    self.net = nn.Sequential(
        nn.Linear(n_embd, n_embd),
        nn.ReLU(),
        # projection layer going back into the residual pathway
        nn.Linear(n_embd, n_embd)
    )

  # computation on a per node level
  def forward(self, x):
    return self.net(x)


# number of heads we'd like is 4
# because n_embd is 32
class BigramLanguageModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)

    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    # create blocks
    # interspersing communication and feedforward many times (3 in this example)
    self.blocks = nn.Sequential(
        Block(n_embd, n_head=4),
        Block(n_embd, n_head=4),
        Block(n_embd, n_head=4)
    )

    # self.sa_heads = MultiHeadAttention(4, n_embd//4) # i.e. 4 heads of 8-dimensional self-attention
    # self.ffwd = FeedForward(n_embd)
    self.lm_head = nn.Linear(n_embd, vocab_size)


  def forward(self, idx, targets=None):
    B, T = idx.shape
    tok_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T, device=device))
    x = tok_emb + pos_emb
    x = self.blocks(x)
    logits = self.lm_head(x) # (B,T,vocab_size)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      idx_cond = idx[:, -block_size:]
      logits, loss = self(idx_cond)
      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1)
      idx = torch.cat((idx, idx_next), dim=1)
    return idx

Dimensionality of input and output is 512 in the paper

Inner layer of the feedforward has dimensionality 2048

Multiplier of 4

So inner layer of the feedforward network needs to be multiplied by 4


In [ ]:
class FeedForward(nn.Module):
  """ a simple linear layer followed by a non-linearity """

  # little feedforward single layer
  # just a linear followed by a relu nonlinearity
  # applying linear on a per token level
  # all the tokens do this independently
  def __init__(self, n_embd):
    super().__init__()
    # add a linear projection
    self.net = nn.Sequential(
        nn.Linear(n_embd, 4 * n_embd),
        nn.ReLU(),
        # projection layer going back into the residual pathway
        nn.Linear(4 * n_embd, n_embd)
    )

  # computation on a per node level
  def forward(self, x):
    return self.net(x)

We start to see a bit of overfitting by observing the training loss and validation loss

Generations still not amazing
Starts to almost look like NASB Bible


In [2]:
# The other innovation for optimising very deep neural networks is:
# add & norm

# residual connection (addition) combined with layer norm
# layer norm is implemented in PyTorch
# based on the paper (Layer Normalization, 2016)
# similar to batch normalisation

# batch normalisation made sure that across the batch dimension
# any individual neuron had unit gaussian distribution
# meaning: zero mean and standard deviation of 1

# copy of batchnorm1d from Andrej Karpathy's makemore series

class BatchNorm1d:

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.momentum = momentum
    self.training = True
    # parameters (trained with backprop)
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
    # buffers (trained with a running 'momentum update)
    self.running_mean = torch.zeros(dim)
    self.running_var = torch.ones(dim)

  def __call__(self, x):
    # calculate the forward pass
    if self.training:
      xmean = x.mean(0, keepdim=True) # batch mean
      xvar = x.var(0, keepdim=True) # batch variance
    else:
      xmean = self.running_mean
      xvar = self.running_var
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    # update the buffers
    if self.training:
      with torch.no_grad():
        self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
        self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = BatchNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape



torch.Size([32, 100])

Guarantees that when we look at the zeroth column it is a zero mean one standard deviation

In [3]:
# normalising every single column of this input
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(7.4506e-09), tensor(1.0000))

The rows are not normalised though

In [4]:
x[0,:].mean(), x[0,:].std() #mean,std of a single input from the batch, of its features

(tensor(0.0411), tensor(1.0431))

Review: we were just normalising columns

In [10]:
# updating batchNorm to layerNorm

# layer norm = row normalised
# batch norm = column normalised

# delete all the buffers as computation does not span across examples

class BatchNorm1d:

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    # we don't need momentum or to care if it's at training time
    # parameters (trained with backprop)
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
    # don't need running buffers

  def __call__(self, x):
    # calculate the forward pass
    # no distinction between training and test time

    # change from 0 to 1
    # this means we don't normalise the columns
    # instead we normalise the rows
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance

    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    # we don't need to maintain any running buffers
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = BatchNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [11]:
x[0,:].mean(), x[0,:].std() #mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

Add and norm position has changed since the transformer paper, there is a pre-norm and post-norm

Updated Block

In [ ]:
class Block(nn.Module):
  """ Transformer block: communication followed by computation """

  # the block basically intersperses communication and then computation
  def __init__(self, n_embd, n_head):
    # n_head is effectively like the group size in group convolution
    # n_embd: embedding dimension, n_head: the number of heads we'd like
    super().__init__()
    head_size = n_embd // n_head
    # communication done using multi-headed self-attention
    self.sa = MultiHeadAttention(n_head, head_size)
    # computation done using the feed forward network on all the tokens independently
    self.ffwd = FeedForward(n_embd)
    # 2 layer norms
    # when the layer norm is normalising our features
    # it behaves as a per token transformation
    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)



  def forward(self, x):
    # skip connection or residual connection
    # x, it forks off to do some communication off the residual highway
    # then joins back to x again

    # pre-norm formulation which differs from post-norm formulation of the transformer paper
    x = x + self.sa(self.ln1(x))
    # x, it forks off to do some computation off the residual highway
    # then joins back to x again
    x = x + self.ffwd(self.ln2(x))
    return x